In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

PATH_TO_DIRECTORY_OF_DATA_TO_SCORE = f'/home/andrew/sleep/data_to_score'
MAPPING_OF_FILE_IDS_TO_CHANNEL_NAME = {
    "SPARC-10-M":"EEG 1", 
    "SPARC-11-M":"EEG 1",
    "SPARC-12-M":"EEG 1",
    "SPARC-13-M":"EEG 1",
    "SPARC-14-M":"EEG 1",
    "SPARC-15-M":"EEG 1",
    "SPARC-17-M":"EEG 2",
    "SPARC-18-M":"EEG 1",
    "SPARC-2-F":"EEG 1",
    "SPARC-4-F":"EEG 1",
    "SPARC-5-F":"Channel-2",
    "SPARC-8-F":"Channel-10",
    "SPARC-9-F":"Channel-2",
    "SPARC-10-F":"Channel-50",
    "SPARC-11-F":"Channel-18",
    "SPARC-12-F":"Channel-42",
    "SPARC-13-F":"EEG 1",
    "SPARC-14-F":"EEG 1",
    "SPARC-15-F":"EEG 1",
    "SPARC-16-F":"EEG 2"
}
IDS = sorted(list(set([id.split('.')[0] for id in os.listdir(PATH_TO_DIRECTORY_OF_DATA_TO_SCORE)])))
print(IDS)

['SPARC-10-F']


In [3]:
from mne.io import read_raw_edf
import torch
# def score_data(id):
id = IDS[0]
zdb_path = f'{PATH_TO_DIRECTORY_OF_DATA_TO_SCORE}/{id}.zdb'
edf_path = f'{PATH_TO_DIRECTORY_OF_DATA_TO_SCORE}/{id}.edf'
if not os.path.exists(zdb_path):
    raise FileNotFoundError(zdb_path)
if not os.path.exists(edf_path):
    raise FileNotFoundError(edf_path)
eeg_channel = MAPPING_OF_FILE_IDS_TO_CHANNEL_NAME[id]
eeg = torch.from_numpy(read_raw_edf(edf_path).get_data(picks=[eeg_channel])[0]).float().reshape(-1,5000)

Extracting EDF parameters from /home/andrew/sleep/data_to_score/SPARC-10-F.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [12]:
PATH_TO_MODEL = f'/home/andrew/sleep/models/gandalfs/gandalf_0/best_model.pt'
from lib.models import Gandalf
model = Gandalf()
model.load_state_dict(torch.load(f=PATH_TO_MODEL,map_location='cpu'))
DEVICE = 'cuda'
model.to(DEVICE);

In [13]:
from torch.utils.data import Dataset,DataLoader
class FeatureSet(Dataset):
    def __init__(self,X):
        self.len = len(X)
        self.X = torch.cat([torch.zeros(4,5000),X,torch.zeros(4,5000)])

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.X[idx:idx+9].flatten()
dataloader = DataLoader(dataset=FeatureSet(eeg),batch_size=32,shuffle=False)

In [14]:
from tqdm import tqdm

In [15]:
with torch.no_grad():
    y_pred = torch.Tensor()
    y_logits = torch.Tensor()
    for Xi in tqdm(dataloader):
        Xi = Xi.to(DEVICE)
        logits = model(Xi)
        y_logits = torch.cat([y_logits,torch.softmax(logits,dim=1).detach().cpu()])
        y_pred = torch.cat([y_pred,torch.softmax(logits,dim=1).argmax(axis=1).detach().cpu()])

  0%|          | 0/1620 [00:00<?, ?it/s]/home/andrew/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:895.)
  return F.conv1d(input, weight, bias, self.stride,
100%|██████████| 1620/1620 [01:03<00:00, 25.41it/s]


In [29]:
y_pred['0']

0        2
1        2
2        2
3        2
4        2
        ..
51818    2
51819    2
51820    2
51821    2
51822    2
Name: 0, Length: 51823, dtype: int64

In [31]:
import pandas as pd
# y_pred = pd.DataFrame(y_pred.numpy()).astype(int)

# y_pred[y_pred['0'] == 0] = 'P'
# y_pred[y_pred['0'] == 1] = 'S'
# y_pred[y_pred['0'] == 2] = 'W'
# y_pred.to_csv(f'{PATH_TO_DIRECTORY_OF_DATA_TO_SCORE}/{id}.csv',index=False) # write to file because inference takes time and you can resume here
y_pred = pd.read_csv(f'{PATH_TO_DIRECTORY_OF_DATA_TO_SCORE}/{id}.csv')

In [23]:
rename_dict = {'W':'Sleep-Wake', 'S':'Sleep-SWS', 'P':'Sleep-Paradoxical', 'X':''}
offset = 10e7       #epoch time period
csv_path = f'{PATH_TO_DIRECTORY_OF_DATA_TO_SCORE}/{id}.csv'

df = pd.read_csv(csv_path)
try:
    conn = sqlite3.connect(zdb_path)
except Error as e:
    print(e)
cur = conn.cursor()

#drop this table - creates issues
query = "DROP TABLE IF EXISTS temporary_scoring_marker;"
cur.execute(query)

#get keyid of scoring
query = "SELECT MAX(id) FROM scoring_revision WHERE name='Machine Data'"
cur.execute(query)
keyid = cur.fetchall()[0][0]

#get starting point for scoring
query = "SELECT id FROM scoring_marker WHERE type LIKE 'Sleep%' AND key_id='"+str(keyid)+"';"
cur.execute(query)
startid = cur.fetchall()[0][0]

#get start time to crreate epochs
query = 'SELECT starts_at FROM scoring_marker WHERE id = '+str(startid)+";"
cur.execute(query)
start_time = cur.fetchall()[0][0]
stop_time = 0

#delete first score before adding machine data
query = "DELETE FROM scoring_marker;"
cur.execute(query)

#insert new epochs with scoring into the table
for i in range(len(df)):
    #calculate epoch
    if i != 0:
        start_time = stop_time
    stop_time = start_time+offset

    score = rename_dict[df.at[i,'0']]
    #insert epoch
    query = f"""
            INSERT INTO scoring_marker 
            (starts_at, ends_at, notes, type, location, is_deleted, key_id)
            VALUES 
            ({start_time}, {stop_time}, '', '{score}', '', 0, {keyid});
            """ 
    cur.execute(query)

conn.commit()
conn.close()

,0
0,2
1,2
2,2
3,2
4,2
...,...
51818,2
51819,2
51820,2
51821,2
